# Сентимент-анализ отзывов на товары

К вашей компании пришел заказчик, которому нужно решение задачи анализа тональности отзывов на товары. Заказчик хочет, чтобы вы оценили возможное качество работы такого алгоритма на небольшой тестовой выборке. При этом больше никаких данных вам не предоставляется. Требуется, чтобы качество работы вашего алгоритма (по accuracy) было строго больше 85%.

Оценка качества в этом задании реализована через контест на Kaggle Inclass:

https://inclass.kaggle.com/c/product-reviews-sentiment-analysis

Вам предстоит посмотреть на предоставленные заказчиком отзывы, собрать похожие отзывы в качестве обучающей выборки, и поэкспериментировать с постановкой задачи (разметкой вашей выборки на позитивные и негативные примеры) так, чтобы результат на примерах заказчика был по возможности получше.

Обратите внимание, что заказчик предоставил всего 100 примеров в качестве тестовой выборки - ситуация, когда размеченных данных почти нет - вообще очень частая в индустриальном анализе данных. Конечно, эти отзывы можно было бы идеально разметить вручную и получить максимальное качество, но вы сами не заинтересованы в таком подходе, т.к. потом придется и на всех новых примерах демонстрировать заказчику идеальную работу, что, конечно, вряд ли будет по силам алгоритму. В любом случае рано или поздно алгоритм придется разрабатывать, поэтому попытки "сжульничать" и не делать никакой модели не одобряются.

# Парсинг

In [29]:
import requests
from bs4 import BeautifulSoup
from random import randint
import csv
from time import sleep

In [118]:
def csv_write(x):
    with open('train.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow((x[0], x[1]))

In [119]:
data = []

In [120]:
url = 'https://torg.mail.ru/review/goods/mobilephones/?page='

In [122]:
for i in range(1, 1200):
    url_i = url + str(i)
    r = requests.get(url_i)
    soup = BeautifulSoup(r.text, 'lxml')
    ads = soup.find_all('div', class_='review-item__body')
    for ad in ads:
        try:
            review = ad.find('div', class_='review-item__content').text.strip()
        except:
            review = ''
        try:
            rating = ad.find('span', class_='review-item__rating-counter').text.strip()
        except:
            rating = ''
            
        csv_write([review, rating])

# Подготовка данных

In [1]:
import pandas as pd
import re

In [2]:
data = pd.read_csv('train.csv', header=None)
data.columns = ['text','rating']

In [3]:
data.rating.value_counts()

5      7895
4,5    3709
4      2581
3      1337
1      1026
3,5     927
2       351
2,5     340
1,5      94
0         4
Name: rating, dtype: int64

За положительный отзыв будем считать с рейтингом 5, за отрицательный - с рейтингом 1

In [4]:
data_train = pd.concat([data[data['rating'] == '1'], data[data['rating'] == '5']])

Негативный отзыв - 0, положительный отзыв - 1

In [5]:
data_train.rating = data_train.rating.replace(['1', '5'], [0, 1])

Посмотрим на сбалансированность классов

In [6]:
data_train.rating.value_counts(normalize=True)

1    0.88499
0    0.11501
Name: rating, dtype: float64

Выборка является несбалансированной. Продублируем для балансировки класс с меткой 0

In [7]:
data_train = pd.concat([data_train[data_train['rating'] == 1], 
                        data_train[data_train['rating'] == 0],
                        data_train[data_train['rating'] == 0],
                        data_train[data_train['rating'] == 0],
                        data_train[data_train['rating'] == 0],
                        data_train[data_train['rating'] == 0],
                        data_train[data_train['rating'] == 0],
                        data_train[data_train['rating'] == 0],
                        data_train[data_train['rating'] == 0]]
                      )

In [8]:
data_train.rating.value_counts(normalize=True)

0    0.509719
1    0.490281
Name: rating, dtype: float64

Выборка сбалансирова

Подготовим текст. Очистим его от небуквенных символов и лишних пробелов. Приведем текст к нижнему регистру

In [9]:
def text_pred(str_text):
    str_text = str_text.replace('.', ' ')
    str_text = str_text.replace('!', ' ')
    str_text = str_text.replace(',', ' ')
    str_text = str_text.replace(';', ' ')
    str_text = str_text.replace(':', ' ')
    str_text = str_text.replace('?', ' ')
    str_text = str_text.replace('+', ' ')
    str_text = str_text.replace('-', ' ')
    str_text = str_text.replace('(', ' ')
    str_text = str_text.replace(')', ' ')
    str_text = str_text.replace('/', ' ')
    str_text = str_text.replace('[', ' ')
    str_text = str_text.replace(']', ' ')
    str_text = str_text.replace('{', ' ')
    str_text = str_text.replace('}', ' ')
    str_text = str_text.replace('=', ' ')
    str_text = str_text.replace('_', ' ')
    str_text = str_text.replace('№', ' ')
    str_text = str_text.replace('#', ' ')
    str_text = str_text.replace('%', ' ')
    str_text = str_text.replace('?', ' ')
    str_text = re.sub('[\n\t\r\s]',' ', str_text)
    str_text = re.sub('Достоинства',' ', str_text)
    str_text = re.sub('Читать полностью',' ', str_text)
    str_text = re.sub('Недостатки',' ', str_text)
    str_text = re.sub('[^а-яА-Я.,\-\s]', ' ', str_text)
    str_text = str_text.lower()
    str_text = re.sub(r'\s+', ' ', str_text)
    
    return str_text

In [10]:
texts = []

for row in data_train.text:
    texts.append(text_pred(row))

In [11]:
labels = data_train.rating

# Построение модели

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cross_validation import cross_val_score

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Обучим pipeline с параметрами по умолчанию

In [13]:
pipeline = Pipeline(steps = [('vect', CountVectorizer()), 
                             ('tfidf', TfidfTransformer()),
                              ('clf', LogisticRegression())])
cross_val_score(pipeline, texts, labels, cv=5, scoring='accuracy').mean()

0.94988719216477269

In [14]:
pipeline.fit(texts, labels)

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

# Предсказание на тестовой выборке

In [25]:
from lxml import etree

f = open("test_file.xml",'w')
f.write('<root>\n')
f2 = open('test.csv','r')
lines = f2.readlines()

for line in lines: 
    f.write(line)
f2.close()
f.write('</root>')
f.close()

f = open('test_file.xml', 'r')
xml = etree.parse(f)
test = xml.findall('review')
test_unicode = [t.text for t in test]

Преобразование данных

In [26]:
test_texts = []
for elem in test_unicode:
    test_texts.append(text_pred(elem))

In [30]:
predict_label = (pipeline.predict_proba(test_texts)[:, 1] > 0.70).astype(int)

In [31]:
sum(predict_label)

50

In [32]:
predict_label = predict_label.tolist()

In [33]:
for i in range(len(predict_label)):
    if predict_label[i] == 0:
        predict_label[i] = 'neg'
    if predict_label[i] == 1:
        predict_label[i] = 'pos'

In [34]:
data_result = []
data_result.append(['Id', 'y'])

for i in range(len(predict_label)):
    data_result.append([i, predict_label[i]])

In [35]:
outfile = open('result.csv', 'w')
writer = csv.writer(outfile)
for row in data_result:
    writer.writerow(row)
outfile.close()